## LLM으로 요약 실습
- `sec03`에서 만든 화자 분리된 전사문 최종 CSV 사용
- SPEAKER 표기를 실제 이름으로 교체
- 회의록 전체를 GPT로 요약해 마크다운으로 저장

In [ ]:
import pandas as pd

# 1) STT+화자 구간이 합쳐진 최종 CSV 로드
meeting_note_csv_path = "../sec03/싼기타_비싼기타_final.csv"
df_rttm = pd.read_csv(meeting_note_csv_path, sep='|')

# 2) 화자 ID를 실제 이름으로 매핑
name_dict = {
    "SPEAKER_00": "이성용",
    "SPEAKER_01": "AI",
}

df_rttm["name"] = df_rttm["speaker_id"].apply(lambda x: name_dict[x])

# 3) 요약 입력을 JSON 텍스트로 변환
meeting_note_txt = df_rttm[['start', 'end', 'name', 'text']].to_json(orient='records', force_ascii=False)

# 4) 요약 프롬프트 구성
system_prompt = f'''
너는 회의 내용을 요약하는 봇이다. 아래 회의록을 읽고, 주요 내용을 요약하라.
결과는 마크다운 형식으로 작성한다.

아래 형식에 맞추어 작성하라.

# 회의 제목

## 주요 내용

## 참석자별 입장

## 결정 사항   

=============== 이하 회의록 ===============

{ meeting_note_txt }
'''

from openai import OpenAI
from dotenv import load_dotenv
import os

# 5) OpenAI 클라이언트 초기화
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# 6) OpenAI API로 요약 생성
response = client.chat.completions.create(
    model='gpt-4o',
    temperature=0.1,
    messages=[
        {"role": "system", "content": system_prompt},
    ]
)

# 7) 결과 저장
summary = response.choices[0].message.content
summary = summary.strip()

print(summary)

with open('./guitar_summary.md', 'w', encoding='utf-8') as f:
    f.write(summary)

# 기타 구매 토론 회의

## 주요 내용
이번 회의에서는 기타를 처음 시작할 때 싼 기타로 시작하는 것이 좋은지, 아니면 비싼 기타로 시작하는 것이 좋은지에 대한 토론이 진행되었습니다. 각 입장에서의 장단점이 논의되었으며, 기타를 선택할 때의 개인적인 상황과 스타일에 맞춰 결정하는 것이 중요하다는 결론에 도달했습니다.

## 참석자별 입장
- **이성용**: 비싼 기타로 시작하는 것이 좋다고 주장. 비싼 기타는 이중 지출을 막을 수 있고, 좋은 기타를 사용함으로써 연습에 대한 책임감과 동기부여가 증가한다고 설명.
- **AI**: 싼 기타로 시작하는 것이 좋다고 주장. 초보자에게는 부담 없이 연습할 수 있는 싼 기타가 적합하며, 실력이 향상되면 비싼 기타로 업그레이드하는 것이 좋다고 설명.

## 결정 사항
- 기타를 처음 시작할 때는 개인의 상황과 스타일에 맞춰 선택하는 것이 중요하다.
- 중요한 것은 기타의 가격보다는 꾸준한 연습과 열정이라는 점에 참석자들이 동의했다.


## LLM으로 교정 실습


In [2]:
#### 필요한 항목만 추출하여 딕셔너리로 변환하기
df_meeting_note = df_rttm[['start', 'end', 'name', 'text']].copy()
df_meeting_note.dropna(inplace=True)

meeting_note_dict = df_meeting_note.to_dict(orient='records')  # 딕셔너리 형태로 변환

# 1) 요약문과 전체 녹취록을 컨텍스트로 삼아 문장 교정
for row in meeting_note_dict:    
    if not isinstance(row['text'], str): # 텍스트가 아닌 다른 형식인 경우 통과
        continue

    correction_system_prompt = f'''
        너는 주어진 회의 녹취록을 수정 및 보완하는 봇이다.
        주어진 회의 녹취록은 STT로 작성된 결과이므로, 이 중에 오류가 있는 부분을 찾아내고 수정하라.

        표현, 말투는 최대한 원본과 일치하도록 유지하되, 잘못 받아적은 내용을 수정하라.
        원본의 내용을 최대한 살리되, 잘못된 내용만 수정하라. 
        어떤 경우에는 A라는 사람이 말한 내용을 B가 말한 것으로 잘못 기록된 경우도 있을 수 있다. 이런 경우에는 문맥을 고려하여 수정하라.

        원본 내용 중 빠지거나, 추가되거나, 잘못 기록된 부분을 찾아내어 수정하라. 수정된 메시지 이외에는 아무것도 작성하지 말라.
        --------------------------------
        회의 요약문 : {summary}
        --------------------------------
        회의 녹취록 전문: {df_meeting_note.to_json(orient='records', force_ascii=False)}
        --------------------------------
        확인할 메시지 원본: {row['text']}
    '''

    response = client.chat.completions.create(
        model='gpt-4o',
        temperature=0.1,
        messages=[
            {"role": "system", "content": correction_system_prompt},
        ]
    )

    correction = response.choices[0].message.content
    if ':' in correction:  # “수정: “ 과 같은 형식으로 반환되는 경우 해결 
        correction = correction.split(':')[-1]
        
    correction = correction.strip() # 좌우 공백 제거

    print(row['name'])
    print(row['text'])
    print(correction)
    print('-----------------')

    row['corrected_text'] = correction

이성용
지금부터 저랑 그 역할극을 합시다 역할극을 스탠딩 코미디 스타일로 할 건데 토론을 하면서 자연스럽게 대화하는 형식으로 하면서 코미디를 진행해봅시다.
그래서 좀 재밌고 자연스럽고 유머러스하게 저랑 대화를 하시면 돼요 자연스럽게
그리고 주제는 쌍기타로 전기기타를 시작하는 게 좋으냐
아니면 비싼 기타로 전기기타를 시작하는 게 좋으냐
이거를 입장을 나눠가지고
저랑 토론해보면 좋을 것 같아요.
두 중에 어떤 역할 맡으실래요?

지금부터 저랑 그 역할극을 합시다. 역할극을 스탠딩 코미디 스타일로 할 건데, 토론을 하면서 자연스럽게 대화하는 형식으로 코미디를 진행해봅시다. 그래서 좀 재밌고 자연스럽고 유머러스하게 저랑 대화를 하시면 돼요. 그리고 주제는 싼 기타로 전기기타를 시작하는 게 좋으냐, 아니면 비싼 기타로 전기기타를 시작하는 게 좋으냐, 이거를 입장을 나눠가지고 저랑 토론해보면 좋을 것 같아요. 두 중에 어떤 역할 맡으실래요?
-----------------
AI
좋습니다. 그럼 제가 싼 기타로 시작하는 게 좋다는 입장을 맡아볼게요. 그럼 성형님은 비싼 기타로 시작하는 게 좋다는 입장이시죠?

좋습니다. 그럼 제가 싼 기타로 시작하는 게 좋다는 입장을 맡아볼게요. 그럼 성용님은 비싼 기타로 시작하는 게 좋다는 입장이시죠?
-----------------
AI
네 맞아요. 준비 되셨나요?

네, 맞아요. 준비되셨나요?
-----------------
이성용
네 됐어요. 시작하시죠.

네, 준비됐어요. 시작하시죠.
-----------------
AI
좋아요.
먼저 3기타로 시작하는게
좋은 이유를 말씀드리겠습니다.
초보자일때는
실수도 많고 기타에 익숙해지는
과정이 필요하니까 비싼 기타보다는
부담없이 연습할 수 있는
싼 기타가 좋다고 봅니다.
무엇보다 실력이 향상되면 그때 좋은 기타로 업그레이드 하는 것도 나쁘지 않죠?
성형님은 어떻게 생각하세요?

좋아요.
먼저 싼 기타로 시작하는 게
좋은 이유를 말씀드리겠습니다.
초보자일 때는
실수도 많고 기타에 익숙해지는


## 결과 저장
- JSON
- Markdown
- Word

In [3]:
import json

# 1) 교정 결과를 JSON으로 저장
with open('./guitar_meeting_note_corrected.json', 'w', encoding='utf-8') as f:
    json.dump(meeting_note_dict, f, ensure_ascii=False, indent=4)

# 2) 마크다운으로 보기 좋게 저장
md_template = ""

for row in meeting_note_dict:
    corrected_text = row.get('corrected_text') or row.get('text', '')
    md_template += f"- **{row.get('name', '')}** : {corrected_text}"

with open('./guitar_meeting_note_corrected.md', 'w', encoding='utf-8') as f:
    f.write(md_template)

%pip install python-docx

from docx import Document

# Word 문서 기본 틀 생성
document = Document()   # docx 객체 생성
document.add_heading('회의록', level=1) # 제목 추가

table = document.add_table(rows=1, cols=2) # 테이블 생성
table.style = 'Table Grid' # 테이블 스타일 지정

hdr_cells = table.rows[0].cells # 첫 번째 행에 셀 추가
hdr_cells[0].text = 'Speaker' # 첫 번째 셀에 Speaker 추가
hdr_cells[1].text = 'Content'   # 두 번째 셀에 Content 추가

# 수정된 녹취록 파일을 읽어들여서 테이블에 추가
for row in meeting_note_dict:
    row_cells = table.add_row().cells
    row_cells[0].text = row.get('name', '')
    row_cells[1].text = row.get('corrected_text') or row.get('text', '')


document.save('./guitar_meeting_note_corrected.docx') # docx 파일 저장

Note: you may need to restart the kernel to use updated packages.
